<a href="https://colab.research.google.com/github/brochowski/ekantorwidget/blob/main/Uczenie_projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cpi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 35.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import sklearn

sklearn.__version__

'1.2.2'

In [79]:
import yfinance as yf
import plotly.express as pxx

# Pobieranie danych dla pary walut USD/PLN
df_USD = yf.download('USDPLN=X', start='2023-06-15', end='2023-12-31', interval='1h')

# Zmiana nazwy kolumny 'USDPLN=X' na bardziej standardową
df_USD.rename(columns={'USDPLN=X': 'Close'}, inplace=True)

df_USD.info()

# Wyświetlanie wykresu
fig = pxx.line(df_USD, x=df_USD.index, y='Close', width=1000, height=500, title='Wykres kursu USD/PLN')
fig.show()

#print(df)

[*********************100%%**********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3380 entries, 2023-06-15 00:00:00+01:00 to 2023-12-29 22:00:00+00:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3380 non-null   float64
 1   High       3380 non-null   float64
 2   Low        3380 non-null   float64
 3   Close      3380 non-null   float64
 4   Adj Close  3380 non-null   float64
 5   Volume     3380 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 184.8 KB


In [34]:
import cpi
import datetime as dt

date_list = []
cpi_list = []
for i in range(12):
    date_list.append(dt.date(2023, i + 1, 1)) # i + 1 to avoid 0 value
    cpi_list.append(cpi.get(date_list[i]))

df = pd.DataFrame({'Date': date_list, 'CPI': cpi_list})

df


,Date,CPI
0,2023-01-01,299.170
1,2023-02-01,300.840
2,2023-03-01,301.836
3,2023-04-01,303.363
4,2023-05-01,304.127
5,2023-06-01,305.109
6,2023-07-01,305.691
7,2023-08-01,307.026
8,2023-09-01,307.789
9,2023-10-01,307.671


In [35]:
df_fedfunds = pd.read_csv('/content/drive/MyDrive/FEDFUNDS.csv', sep=',')
fedfunds = df_fedfunds.copy()
#fedfunds
df_fedfunds['DATE'] = pd.to_datetime(df_fedfunds['DATE']) # konwersja kolumny na datę
fundsUSA = df_fedfunds[df_fedfunds['DATE'].dt.year == 2023]
df = pd.DataFrame({'Date': date_list, 'CPI_USA': cpi_list, 'FUNDS_USA': fundsUSA['FEDFUNDS']})
df

,Date,CPI_USA,FUNDS_USA
822,2023-01-01,299.170,4.33
823,2023-02-01,300.840,4.57
824,2023-03-01,301.836,4.65
825,2023-04-01,303.363,4.83
826,2023-05-01,304.127,5.06
827,2023-06-01,305.109,5.08
828,2023-07-01,305.691,5.12
829,2023-08-01,307.026,5.33
830,2023-09-01,307.789,5.33
831,2023-10-01,307.671,5.33


In [36]:
df_cp=df.copy()
df_cpi_pl = pd.read_csv('/content/drive/MyDrive/inflacja.csv', sep=';')
cpi_pl = df_cpi_pl.copy()
cpi_pl.drop(['label'], axis=1, inplace=True)
cpi_pl.drop(['infl_2021'], axis=1, inplace=True)
cpi_pl.drop(['infl_2022'], axis=1, inplace=True)
df_cp['temp_index'] = range(len(df_cp))
#df_cp['funds_pl'] = range(len(df_cp))
cpi_pl['temp_index'] = range(len(cpi_pl))
cpi_pl['funds_pl'] = range(len(cpi_pl))
df_new = pd.merge(df_cp, cpi_pl[['temp_index', 'infl_2023']], how='left', on='temp_index')
df_new = pd.merge(df_new, cpi_pl[['temp_index', 'Stopy_2023']], how='left', on='temp_index')
df_new.drop(['temp_index'], axis=1, inplace=True)
#df_new.drop(['funds_pl'], axis=1, inplace=True)
df_new.rename(columns={'infl_2023': 'CPI_PL'}, inplace=True)
df_new.rename(columns={'Stopy_2023': 'FUNDS_PL'}, inplace=True)
df_new
#df = pd.merge(df, cpi_pl['2023'], how='inner')
#df
#df = pd.DataFrame({'Date': date_list, 'CPI_USA': cpi_list, 'FUNDS_USA': fundsUSA['FEDFUNDS'], 'CPI_PL': cpi_pl['2023']})
#df

,Date,CPI_USA,FUNDS_USA,CPI_PL,FUNDS_PL
0,2023-01-01,299.170,4.33,16.6,6.75
1,2023-02-01,300.840,4.57,18.4,6.75
2,2023-03-01,301.836,4.65,16.1,6.75
3,2023-04-01,303.363,4.83,14.7,6.75
4,2023-05-01,304.127,5.06,13.0,6.75
5,2023-06-01,305.109,5.08,11.5,6.75
6,2023-07-01,305.691,5.12,10.8,6.00
7,2023-08-01,307.026,5.33,10.1,5.75
8,2023-09-01,307.789,5.33,8.2,5.75
9,2023-10-01,307.671,5.33,6.6,5.75


In [67]:

def get_row_by_month(dframe, month):
    for ind in dframe.index:
      date = dframe['Date'][ind]
      if int(date.month) == month:
        row = {'DATE' : dframe['Date'][ind], 'CPI_USA' : dframe['CPI_USA'][ind], 'FUNDS_USA' : dframe['FUNDS_USA'][ind], 'CPI_PL' : dframe['CPI_PL'][ind], 'FUNDS_PL' : dframe['FUNDS_PL'][ind]}
        return row
    return None

#print(get_row_by_month(df_new, 4)['CPI_USA'])

303.363


In [111]:
from datetime import datetime

def konwertuj_na_date(data_str):
    maska = "%Y-%m-%d"
    data = datetime.strptime(data_str, maska)
    return data

fed_dates=[konwertuj_na_date('2023-01-31').date(), konwertuj_na_date('2023-03-21').date(), konwertuj_na_date('2023-05-02').date(), konwertuj_na_date('2023-06-13').date(), konwertuj_na_date('2023-07-25').date(), konwertuj_na_date('2023-09-19').date(), konwertuj_na_date('2023-10-31').date(), konwertuj_na_date('2023-12-12').date(), konwertuj_na_date('2024-01-31').date()]
nbp_dates=[konwertuj_na_date('2023-01-03').date(), konwertuj_na_date('2023-02-07').date(), konwertuj_na_date('2023-03-07').date(), konwertuj_na_date('2023-04-04').date(), konwertuj_na_date('2023-05-09').date(), konwertuj_na_date('2023-06-05').date(), konwertuj_na_date('2023-07-05').date(), konwertuj_na_date('2023-08-21').date(), konwertuj_na_date('2023-09-05').date(), konwertuj_na_date('2023-10-03').date(), konwertuj_na_date('2023-11-07').date(), konwertuj_na_date('2023-12-05').date(), konwertuj_na_date('2024-01-08').date()]
pl_election_dates=[konwertuj_na_date('2023-10-15').date(), konwertuj_na_date('2024-07-04').date()]

def find_next_fed_date(date, meeting_dates):
  meeting_dates.sort()
  for i in range(len(meeting_dates)):
    if meeting_dates[i] >= date:
      return meeting_dates[i]

df_money = df_USD.copy()
for index in df_money.index:
  df_money.at[index, 'CPI_USA'] = get_row_by_month(df_new, index.month)['CPI_USA']
  df_money.at[index, 'FUNDS_USA'] = get_row_by_month(df_new, index.month)['FUNDS_USA']
  df_money.at[index, 'CPI_PL'] = get_row_by_month(df_new, index.month)['CPI_PL']
  df_money.at[index, 'FUNDS_PL'] = get_row_by_month(df_new, index.month)['FUNDS_PL']
  df_money.at[index, 'DAY_OF_WEEK'] = index.day_of_week
  df_money.at[index, 'MONTH'] = index.month
  df_money.at[index, 'HOUR'] = index.hour
  df_money.at[index, 'DAYS_TO_FED'] = (find_next_fed_date(index.date(), fed_dates)-index.date()).days
  df_money.at[index, 'DAYS_TO_NBP'] = (find_next_fed_date(index.date(), nbp_dates)-index.date()).days
  df_money.at[index, 'DAYS_TO_ELECTIONS_PL'] = (find_next_fed_date(index.date(), pl_election_dates)-index.date()).days
df_money




,Open,High,Low,Close,Adj Close,Volume,CPI_USA,FUNDS_USA,CPI_PL,FUNDS_PL,DAY_OF_WEEK,MONTH,HOUR,DAYS_TO_FED,DAYS_TO_NBP,DAYS_TO_ELECTIONS_PL
Datetime,,,,,,,,,,,,,,,,
2023-06-15 00:00:00+01:00,4.10910,4.10920,4.0992,4.10332,4.10332,0,305.109,5.08,11.5,6.75,3.0,6.0,0.0,40.0,20.0,122.0
2023-06-15 01:00:00+01:00,4.10332,4.10870,4.0987,4.10470,4.10470,0,305.109,5.08,11.5,6.75,3.0,6.0,1.0,40.0,20.0,122.0
2023-06-15 02:00:00+01:00,4.10370,4.11795,4.1008,4.11672,4.11672,0,305.109,5.08,11.5,6.75,3.0,6.0,2.0,40.0,20.0,122.0
2023-06-15 03:00:00+01:00,4.11300,4.11757,4.1107,4.11310,4.11310,0,305.109,5.08,11.5,6.75,3.0,6.0,3.0,40.0,20.0,122.0
2023-06-15 04:00:00+01:00,4.11300,4.11624,4.1107,4.11480,4.11480,0,305.109,5.08,11.5,6.75,3.0,6.0,4.0,40.0,20.0,122.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29 18:00:00+00:00,3.92677,3.93154,3.9248,3.92913,3.92913,0,306.746,5.33,6.2,5.75,4.0,12.0,18.0,33.0,10.0,188.0
2023-12-29 19:00:00+00:00,3.92951,3.93514,3.9278,3.93130,3.93130,0,306.746,5.33,6.2,5.75,4.0,12.0,19.0,33.0,10.0,188.0
2023-12-29 20:00:00+00:00,3.93177,3.93453,3.9261,3.93230,3.93230,0,306.746,5.33,6.2,5.75,4.0,12.0,20.0,33.0,10.0,188.0
